In [20]:
# CONNECT
from qdrant_client import QdrantClient,models
client=QdrantClient(url="http://localhost:6333")

In [21]:
my_collection="search_dataset_cosine"

In [29]:
#TẠO COLLECTION

#client.delete_collection(collection_name=my_collection)
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
    hnsw_config=models.HnswConfigDiff(
        m=32,               #  số lượng cạnh tối đa mà mỗi nút trong đồ thị HNSW có thể có
                            #Điều này giúp tăng cường kết nối giữa các nút=>tìm kiếm nhanh hơn và chính xác hơn, nhưng cũng làm tăng kích thước và phức tạp của đồ thị.
        ef_construct=200,  # cải thiện độ chính xác của việc tìm kiếm bằng cách xem xét nhiều hàng xóm hơn khi xây dựng đồ thị,
                            #nhưng cũng làm tăng thời gian và tài nguyên cần thiết để xây dựng đồ thị. 
    )
)




UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `search_dataset_cosine` already exists!"},"time":0.004174068}'

In [30]:
# LOAD DATASET to COLLECTION
import json
dataset_file=open('dataset.json') #path file dataset
dataset=json.load(dataset_file)
print(dataset[0])
print(len(dataset[0]["vector"])) #768
len(dataset)

{'id': 'ID', 'so-hieu': 'SoHieu', 'loai-van-ban': 'LoaiVanBan', 'noi-ban-hanh': 'NoiBanHanh', 'nguoi-ky': 'NguoiKy', 'ngay-ban-hanh': 'NgayBanHanh', 'ngay-hieu-luc': 'NgayHieuLuc', 'ngay-cong-bao': 'NgayCongBao', 'so-cong-bao': 'SoCongBao', 'tinh-trang': 'TinhTrang', 'chuong': 'Chuong', 'muc': 'Muc', 'dieu': 'Dieu', 'vector': [0.009931422770023346, 0.12482008337974548, -0.04039105772972107, -1.0201289653778076, -0.10611715912818909, -0.3670913577079773, 0.642419159412384, -0.4343612790107727, -0.20189569890499115, 0.288735568523407, 0.12281358242034912, -0.20548969507217407, -0.02293645776808262, 0.22771745920181274, 0.3995859920978546, 0.057859718799591064, -0.13268005847930908, -0.23242363333702087, -0.19131219387054443, -0.6911299228668213, 0.33153215050697327, -1.2840988636016846, 0.1636037975549698, -0.7262627482414246, 0.4323790669441223, -0.22903355956077576, 0.18935872614383698, 0.04061983525753021, -0.9065213203430176, 0.6014841794967651, 0.4398919641971588, 0.5618135929107666

1000

In [25]:
client.upload_points(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=idx,vector=doc["vector"],payload=doc
        )for idx,doc in enumerate(dataset)
    ],
)

SEARCH

In [33]:

# search toàn bộ văn bản
def search_full(query_vector, limit):
   my_collection="search_dataset_cosine"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit
   )
   for i in search_result:
      print(i)


# thêm filter
def search_with_filter(query_vector, limit):
   my_collection="search_dataset_cosine"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      query_filter=models.Filter(
         must=[models.FieldCondition(key="dieu", match=models.MatchValue(value="2"))]
      ),
      limit=limit
   )

 
   return search_result

# chỉ in ra 1 số thông tin cần thiết
import json
import pandas as pd
df=pd.read_csv('data.csv')
data_csv=df.to_numpy()
#print(data) #(71727, 14)


def search_with_some_inf(query_vector, limit):
   global data_csv
   my_collection="search_dataset_cosine"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit,
      search_params=models.SearchParams(
                exact=True,),  # Turns on the exact search mode
   )
   #print question
   if len(search_result)==0:
      print("Không có thông tin tìm kiếm")
   else:
      print("Kết quả thông tin tìm kiếm là:")
      answer_array=[]
      for i in search_result:

         answer_json={
            'index':i.id,
            'score':i.score,
            'noi dung':data_csv[i.id-1,-1],
            'id_van_ban':i.payload['id'],
            'so-hieu':i.payload['so-hieu'],
            'loai-van-ban':i.payload['loai-van-ban'],
            'noi-ban-hanh':i.payload['noi-ban-hanh'],
            'nguoi-ky':i.payload['nguoi-ky'],
            'ngay-ban-hanh':i.payload['ngay-ban-hanh'],
            'so-cong-bao':i.payload['so-cong-bao'],
            'tinh-trang':i.payload['tinh-trang'],
            'chuong':i.payload['chuong'],
            'muc':i.payload['muc'],
            'dieu':i.payload['dieu'],
            
            
         }
         if len(answer_array)==0:
            answer_array.append(answer_json)
         else:
            count=0
            for answer in answer_array:
            #id là của 1 văn bản , phải thêm chương, điều, mục
               if (answer_json['index']==answer['index'] and
                  answer_json['id_van_ban']==answer['id_van_ban'] and 
                  answer_json['chuong']==answer['chuong'] and
                  answer_json['muc']==answer['muc'] and
                  answer_json['dieu']==answer['dieu'] ):
                  count=count+1
            if count==0:
               answer_array.append(answer_json)

     
      
      for i in answer_array:
         print(i)



In [34]:
# THỬ  VỚI 1 QUERY VECTOR CÓ SẴN TRONG DATASET
query_vector=dataset[10]
#print(query_vector)
print(data_csv[10-1])
#print(type(query_vector))
print("\nResult:")
search_with_some_inf(query_vector['vector'],3)


[1121 '987/QĐ-BVHTTDL' 'Quyết định' 'Bộ Văn hoá, Thể thao và du lịch'
 'Tạ Quang Đông' '11/04/2024' '11/04/2024' 'Đang cập nhật' 'Đang cập nhật'
 'Còn hiệu lực' 0 0 3
 '  Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ ']

Result:
Kết quả thông tin tìm kiếm là:
{'index': 10, 'score': 1.0, 'noi dung': '  Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ ', 'id_van_ban': '1121', 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Văn hoá, Thể thao và du lịch', 'nguoi-ky': 'Tạ Quang Đông', 'ngay-ban-hanh': '11/04/2024', 'so-cong-bao': 'Đang cập nhật', 'tinh-trang': 'Còn hiệu lực', 'chuong': '0', 'muc': '0', 'dieu': '3'}
{'index': 7, 'score': 1.0, 'noi dung': ' Thủ tục hành chính nội bộ được sửa đổi  bổ sung', 'id_van_ban': '1121', 'so-hieu': '987/QĐ-BVHT

In [36]:
# THỬ VỚI 1 CÂU TEXT CÓ TÁCH
import torch
from vector_cau_hoi_bangoc import text_to_vector_co_tach
text = '''Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ;
Hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ  của các Khối thi đua gửi trực tiếp hoặc qua đường bưu điện hoặc trực tuyến (hồ sơ điện tử) đến Bộ Văn hóa  Thể thao và Du lịch (qua Vụ Tổ chức cán bộ)'''
query_vector=text_to_vector_co_tach(text).tolist() #<class 'numpy.ndarray'>  => list

#print(query_vector.shape) #(2, 768)
print(len(query_vector))

# search mỗi câu với 1 kết quả tương ứng
for i in range(len(query_vector)):
 print(f"Top Result of câu {i+1}")
 answer_array=search_with_some_inf(query_vector[i],3)



['Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ', 'Hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ  của các Khối thi đua gửi trực tiếp hoặc qua đường bưu điện hoặc trực tuyến (hồ sơ điện tử) đến Bộ Văn hóa  Thể thao và Du lịch (qua Vụ Tổ chức cán bộ)']
Đang xử lý line =  Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ
Đang xử lý line =  Hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ  của các Khối thi đua gửi trực tiếp hoặc qua đường bưu điện hoặc trực tuyến (hồ sơ điện tử) đến Bộ Văn hóa  Thể thao và Du lịch (qua Vụ Tổ chức cán bộ)
2
Top Result of câu 1
Kết quả thông tin tìm kiếm là:
{'index': 122, 'score': 0.66558987, 'noi dung': ' phúc lợi xã hội', 'id_van_ban': '1121', 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Văn hoá, Thể th

In [37]:
# THỬ VỚI TEXT KHÔNG TÁCH
import torch
from vector_cau_hoi_bangoc import text_to_vector_ko_tach
text = 'Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ'

query_vector=text_to_vector_ko_tach(text)[0].tolist() #<class 'numpy.ndarray'>  => list

#print(query_vector.shape) #(2, 768)
print(len(query_vector))

# search mỗi câu với 1 kết quả tương ứng
answer_array=search_with_some_inf(query_vector,3)

['Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ']
Đang xử lý line =  Khối trưởng các Khối thi đua gồm các cơ quan  đơn vị thuộc Bộ có trách nhiệm tổ chức họp bình xét  lập hồ sơ đề nghị xét tặng danh hiệu  Cờ thi đua của Bộ
768
Kết quả thông tin tìm kiếm là:
{'index': 122, 'score': 0.66558987, 'noi dung': ' phúc lợi xã hội', 'id_van_ban': '1121', 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-hanh': 'Bộ Văn hoá, Thể thao và du lịch', 'nguoi-ky': 'Tạ Quang Đông', 'ngay-ban-hanh': '11/04/2024', 'so-cong-bao': 'Đang cập nhật', 'tinh-trang': 'Còn hiệu lực', 'chuong': '0', 'muc': '0', 'dieu': '10'}
{'index': 120, 'score': 0.64258206, 'noi dung': ' Giá trị\xa0tổng\xa0sản lượng  doanh thu  lợi nhuận  tỷ suất lợi nhuận  nộp ngân sách  đầu tư tái sản xuất  thu nhập bình quân', 'id_van_ban': '1121', 'so-hieu': '987/QĐ-BVHTTDL', 'loai-van-ban': 'Quyết định', 'noi-ban-h

TINH CHỈNH SEARCH

In [ ]:
def search_finetune(query_vector,limit):
   global data_csv
   my_collection="search_dataset"
   search_result=client.search(
      collection_name=my_collection,
      query_vector=query_vector,
      limit=limit,
      
   )
   #print question
   if len(search_result)==0:
      print("Không có thông tin tìm kiếm")
   else:
      print("Kết quả thông tin tìm kiếm là:")
      answer_array=[]
      for i in search_result:

         answer_json={
            'index':i.id,
            'score':i.score,
            'noi dung':data_csv[i.id-1,-1],
            'id_van_ban':i.payload['id'],
            'so-hieu':i.payload['so-hieu'],
            'loai-van-ban':i.payload['loai-van-ban'],
            'noi-ban-hanh':i.payload['noi-ban-hanh'],
            'nguoi-ky':i.payload['nguoi-ky'],
            'ngay-ban-hanh':i.payload['ngay-ban-hanh'],
            'so-cong-bao':i.payload['so-cong-bao'],
            'tinh-trang':i.payload['tinh-trang'],
            'chuong':i.payload['chuong'],
            'muc':i.payload['muc'],
            'dieu':i.payload['dieu'],
            
            
         }
         if len(answer_array)==0:
            answer_array.append(answer_json)
         else:
            count=0
            for answer in answer_array:
            #id là của 1 văn bản , phải thêm chương, điều, mục
               if (answer_json['index']==answer['index']):
                  count=count+1
            if count==0:
               answer_array.append(answer_json)

     
      
      for i in answer_array:
         print(i)